In [16]:
import glob
import os
import shutil
from tqdm import tqdm

PATH_TO_IMAGES = '/home/vlad/datasets/drons_vs_berds/images'
PATH_TO_LABELS = '/home/vlad/datasets/drons_vs_berds/coco/labels'
PATH_TO_SAVE_DATASET = '/home/vlad/datasets/drons_vs_berds/coco/dataset'

tests_videos=[
    "00_06_10_to_00_06_27",
    "2019_08_19_GP015869_1520_inspire" ,
    "GOPR5843_002",
    "dji_phantom_4_mountain_hover",
    "gopro_002",
    "parrot_disco_zoomin_zoomout",
    "two_distant_phantom"
]
valids_videos = [
    "00_10_09_to_00_10_40", 
    "2019_08_19_GOPR5869_1530_phantom", 
    "2019_09_02_C0002_3700_mavic", 
    "GOPR5846_002", 
    "GOPR5847_004", 
    "custom_fixed_wing_1", 
    "distant_parrot_2", 
    "dji_matrice_210_sky", 
    "dji_mavick_mountain", 
    "fixed_wing_over_hill_2", 
    "gopro_005", 
    "gopro_008",       
    "off_focus_parrot_birds", 
    "parrot_disco_distant_cross_3",
]

train_videos = list(map(lambda path:  path.split('/')[-2]  , glob.glob(os.path.join(PATH_TO_IMAGES,"*/" ))  ))   
train_videos  = [video for video in train_videos if (video not in tests_videos) & (video not in valids_videos)]

paths_to_tests_videos  = [os.path.join(PATH_TO_LABELS,f'{name_video}.json')  for name_video  in tests_videos]
paths_to_valids_videos  = [os.path.join(PATH_TO_LABELS,f'{name_video}.json')  for name_video  in valids_videos]
paths_to_train_videos  = [os.path.join(PATH_TO_LABELS,f'{name_video}.json')  for name_video  in train_videos]

In [17]:
import json


def coco_merge(paths_to_ann, output_file) :
    """
        Merge COCO annotation json.
        source - https://gradiant.github.io/pyodi/reference/apps/coco-merge/
    """
 
    annotations = []
    for path_to_ann in paths_to_ann:
        
        with open(path_to_ann, "r") as f:
            annotations+=[json.load(f)]  
    data_extend = annotations[0]
    output  = {
        k: data_extend[k] for k in data_extend if k not in ("images", "annotations")
    }

    output["images"], output["annotations"] = [], []

    for i, data in enumerate(annotations):

        cat_id_map = {}
        for new_cat in data["categories"]:
            new_id = None
            for output_cat in output["categories"]:
                if new_cat["name"] == output_cat["name"]:
                    new_id = output_cat["id"]
                    break

            if new_id is not None:
                cat_id_map[new_cat["id"]] = new_id
            else:
                new_cat_id = max(c["id"] for c in output["categories"]) + 1
                cat_id_map[new_cat["id"]] = new_cat_id
                new_cat["id"] = new_cat_id
                output["categories"].append(new_cat)

        img_id_map = {}
        for image in data["images"]:
            n_imgs = len(output["images"])
            img_id_map[image["id"]] = n_imgs
            image["id"] = n_imgs

            output["images"].append(image)

        for annotation in data["annotations"]:
            n_anns = len(output["annotations"])
            annotation["id"] = n_anns
            annotation["image_id"] = img_id_map[annotation["image_id"]]
            annotation["category_id"] = cat_id_map[annotation["category_id"]]

            output["annotations"].append(annotation)

    with open(output_file, "w") as f:
        json.dump(output, f)

    return output

In [20]:
print(len(coco_merge(paths_to_valids_videos, os.path.join(PATH_TO_SAVE_DATASET,'drons_vs_berds_valid.json'))['images']))
print(len(coco_merge(paths_to_tests_videos, os.path.join(PATH_TO_SAVE_DATASET,'drons_vs_berds_test.json')   )['images']))
print(len(coco_merge(paths_to_train_videos, os.path.join(PATH_TO_SAVE_DATASET,'drons_vs_berds_train.json')   )['images']))

7805
3116
41632


In [21]:
anns = coco_merge(paths_to_tests_videos, os.path.join(PATH_TO_SAVE_DATASET,'drons_vs_berds_test.json') )

In [22]:


import cv2
for index, image in enumerate(anns['images'])  :
 
    image_id = image['id'] 
    file_name = image['file_name']  
    path_to_iamge = os.path.join(PATH_TO_IMAGES, file_name ) 
    image = cv2.imread(path_to_iamge)

    for ann in anns['annotations']:
        if ann['image_id'] == image_id:
            [x,y,w,h] = ann['bbox']
            cv2.rectangle(image, (int(x), int(y)),  (int(x+w), int(y+h)), (255,0,0), 1) 
     
           
    path_save = os.path.join('/home/vlad/datasets/drons_vs_berds/coco/testing_ann', file_name )

    print(path_save)
    os.makedirs(os.path.dirname(path_save),exist_ok=True)
    cv2.imwrite( path_save, image)
    # plt.show()

/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0000.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0002.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0004.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0006.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0008.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0010.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0012.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0014.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0016.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0018.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0020.jpg
/home/vlad/datasets/drons_vs_berds/coco/testing_ann/00_06_10_to_00_06_27/0022.jpg
/home/vlad/datas